# Objective:

Scraping of [books.toscrape.com](https://books.toscrape.com) with 'BeautifulSoup4' and 'Requests', get all the data includes:

* category
* code
* cover
* title
* rating
* price (excl. tax)
* price (incl. tax)
* tax
* stock status
* number of stock available
* description
* number of reviews

In [1]:
# --- Install Dependencies ---
!pip install requests beautifulsoup4 pandas tqdm

# Import Libraries

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

# Scraping Process

In [13]:
# Base URL
BASE_URL = "https://books.toscrape.com/"

# Function to get soup object
def get_soup(url):
    response = requests.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.text, "html.parser")

In [14]:
# Get all categories
def get_categories():
    soup = get_soup(BASE_URL)
    categories = {}
    cat_section = soup.find('ul', class_='nav-list').find('ul').find_all('a')
    for cat in cat_section:
        name = cat.text.strip()
        link = BASE_URL + cat['href']
        categories[name] = link
    return categories

In [15]:
# Extract book details from book page
def get_book_details(book_url, category):
    soup = get_soup(book_url)

    # Extract product info table
    table = soup.find('table', class_='table table-striped')
    data = {row.th.text.strip(): row.td.text.strip() for row in table.find_all('tr')}

    # Extract other fields
    title = soup.find('div', class_='product_main').h1.text.strip()
    rating = soup.find('p', class_='star-rating')['class'][1]
    price_excl = data.get('Price (excl. tax)')
    price_incl = data.get('Price (incl. tax)')
    tax = data.get('Tax')
    upc = data.get('UPC')
    stock_text = data.get('Availability', '')
    stock_number = ''.join(filter(str.isdigit, stock_text))
    description_tag = soup.select_one('#product_description + p')
    description = description_tag.text.strip() if description_tag else ''
    num_reviews = data.get('Number of reviews', '0')
    cover = BASE_URL + soup.find('div', class_='item active').img['src'].replace('../', '')

    return {
        'category': category,
        'code (UPC)': upc,
        'cover': cover,
        'title': title,
        'rating': rating,
        'price (excl. tax)': price_excl,
        'price (incl. tax)': price_incl,
        'tax': tax,
        'stock status': 'In stock' if 'In stock' in stock_text else 'Out of stock',
        'number of stock available': stock_number,
        'description': description,
        'number of reviews': num_reviews
    }


In [16]:
# Scrape all books from category pages
def scrape_books_in_category(category_name, category_url):
    books = []
    page_url = category_url

    while True:
        soup = get_soup(page_url)
        book_links = [BASE_URL + 'catalogue/' + a['href'].replace('../../../', '') for a in soup.select('h3 a')]

        for link in book_links:
            try:
                books.append(get_book_details(link, category_name))
                time.sleep(0.2)
            except Exception as e:
                print(f"Error on {link}: {e}")
                continue

        # Next page
        next_page = soup.select_one('li.next > a')
        if next_page:
            next_href = next_page['href']
            page_url = category_url.rsplit('/', 1)[0] + '/' + next_href
        else:
            break
    return books

In [17]:
# Main Scraping Process
all_books = []
categories = get_categories()

print(f"Found {len(categories)} categories.\n")

for cat_name, cat_url in tqdm(categories.items(), desc="Scraping categories"):
    print(f"\n--- Scraping Category: {cat_name} ---")
    books_in_cat = scrape_books_in_category(cat_name, cat_url)
    all_books.extend(books_in_cat)

Found 50 categories.



Scraping categories:   0%|          | 0/50 [00:00<?, ?it/s]


--- Scraping Category: Travel ---


Scraping categories:   2%|▏         | 1/50 [00:03<03:05,  3.78s/it]


--- Scraping Category: Mystery ---


...

Scraping categories:  96%|█████████▌| 48/50 [05:42<00:01,  1.15it/s]


--- Scraping Category: Erotica ---


Scraping categories:  98%|█████████▊| 49/50 [05:42<00:00,  1.33it/s]


--- Scraping Category: Crime ---


Scraping categories: 100%|██████████| 50/50 [05:43<00:00,  6.87s/it]


In [18]:
# Save to CSV
df = pd.DataFrame(all_books)
df.to_csv('books_data.csv', index=False)
print("\n✅ Scraping completed! Data saved to 'books_data.csv'.")


✅ Scraping completed! Data saved to 'books_data.csv'.


In [19]:
# Display sample data
df.head()

,category,code (UPC),cover,title,rating,price (excl. tax),price (incl. tax),tax,stock status,number of stock available,description,number of reviews
0,Travel,a22124811bfa8350,https://books.toscrape.com/media/cache/6d/41/6...,It's Only the Himalayas,Two,Â£45.17,Â£45.17,Â£0.00,In stock,19,"âWherever you go, whatever you do, just . . ...",0
1,Travel,ce60436f52c5ee68,https://books.toscrape.com/media/cache/fe/8a/f...,Full Moon over Noahâs Ark: An Odyssey to Mou...,Four,Â£49.43,Â£49.43,Â£0.00,In stock,15,Acclaimed travel writer Rick Antonson sets his...,0
2,Travel,f9705c362f070608,https://books.toscrape.com/media/cache/c7/1a/c...,See America: A Celebration of Our National Par...,Three,Â£48.87,Â£48.87,Â£0.00,In stock,14,To coincide with the 2016 centennial anniversa...,0
3,Travel,1809259a5a5f1d8d,https://books.toscrape.com/media/cache/ca/30/c...,Vagabonding: An Uncommon Guide to the Art of L...,Two,Â£36.94,Â£36.94,Â£0.00,In stock,8,With a new foreword by Tim Ferriss â¢Thereâ...,0
4,Travel,a94350ee74deaa07,https://books.toscrape.com/media/cache/45/21/4...,Under the Tuscan Sun,Three,Â£37.33,Â£37.33,Â£0.00,In stock,7,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...,0
